In [1]:
from datahandler import DataHandler
from models.unet import *
from models.unet_se import *
from models.resnet_fcn import *
from models.resnet_se_fcn import *
from models.resnet_fcn import *
from models.vgg19_fcn import *
from models.vgg19_se_fcn import *
from generator import *
from params import *
from callbacks import getCallbacks

from tqdm import tqdm
import os
import skimage.io as io

from keras.models import *
from keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

import argparse
import sys

Using TensorFlow backend.


In [2]:
#set common variables
epochs = 25
batch_size = 32
verbose = 1


resetSeed()

In [3]:
#Get data and generators
dh = DataHandler()
tr_images, tr_masks, te_images, te_masks = dh.getData()

Validation data: 100%|██████████████████████████| 43/43 [00:43<00:00,  1.01s/it]


In [4]:
model = getVGG19FCN()
model.load_weights('logs/vgg19FCN/kfold_vgg19FCN/kfold_vgg19FCN_dice_DA_K2/kfold_vgg19FCN_dice_DA_K2_weights.h5'

SyntaxError: unexpected EOF while parsing (<ipython-input-4-70fe2624a794>, line 2)

In [ ]:
plt.gray()

plt.imshow(np.squeeze(tr_images[104] + tr_masks[104]))

In [ ]:
def getGenerator(images, bs=1):
    image_datagen = ImageDataGenerator(rescale=1./255)
    image_datagen.fit(images, augment = True)
    image_generator = image_datagen.flow(x = images, batch_size=bs,
            shuffle = False)

    return image_generator

image, hdr = dh.getImageData('./data/kfold_data/images/0.nii')
gt_mask, _ = dh.getImageData('./data/kfold_data/masks/0.nii', is_mask=True)
    
g = getGenerator(image)
r = model.predict_generator(g, steps=len(image))

print(r.shape)

plt.imshow(np.squeeze(r[24] * 255))